# Classification - SQL, Pandas, Matplotlib - Wildfire Database

Add Field to summarize causes - Update Prediction Model to tell if: Natural, Man-Made, or Unknown...??

Should Use Lambda function, but I know iLoc was used before... Compare and Document Methods

Get Dummies for 


Bar Chart -  Count Totals by Cause (or year for Reviews DB), and use in Matplotlib


In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("FireSQLdb.sqlite")
cur = conn.cursor()
cur

In [3]:
# No Cause CODE
fire_qry = '''
SELECT f.FIRE_YEAR as FireYear,
    f.DISCOVERY_DOY as DiscoveryDOY,
    f.STAT_CAUSE_DESCR  as FireCause,
    f.FIRE_SIZE as FireSize,
    f.OWNER_DESCR as LandOwnerType,
    f.STATE as FireState
FROM Fires  f;
'''

In [ ]:
fire_qry = '''
SELECT f.FIRE_YEAR as FireYear,
    f.DISCOVERY_DOY as DiscoveryDOY,
    f.STAT_CAUSE_CODE  as FireCode,
    f.STAT_CAUSE_DESCR  as FireCause,
    f.FIRE_SIZE as FireSize,
    f.OWNER_DESCR as LandOwnerType,
    f.STATE as FireState
FROM Fires  f;
'''

In [4]:
df_fire = pd.read_sql_query(fire_qry, conn)

In [5]:
df_fire

,FireYear,DiscoveryDOY,FireCause,FireSize,LandOwnerType,FireState
0,2005,33,Miscellaneous,0.10,USFS,CA
1,2004,133,Lightning,0.25,USFS,CA
2,2004,152,Debris Burning,0.10,STATE OR PRIVATE,CA
3,2004,180,Lightning,0.10,USFS,CA
4,2004,180,Lightning,0.10,USFS,CA
...,...,...,...,...,...,...
1880460,2015,269,Missing/Undefined,0.01,STATE OR PRIVATE,CA
1880461,2015,278,Miscellaneous,0.20,MUNICIPAL/LOCAL,CA
1880462,2015,122,Missing/Undefined,0.10,MUNICIPAL/LOCAL,CA
1880463,2015,287,Missing/Undefined,2.00,MUNICIPAL/LOCAL,CA


In [6]:
df_fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   FireYear       int64  
 1   DiscoveryDOY   int64  
 2   FireCause      object 
 3   FireSize       float64
 4   LandOwnerType  object 
 5   FireState      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 86.1+ MB


In [7]:
df_fire.notnull().sum()

FireYear         1880465
DiscoveryDOY     1880465
FireCause        1880465
FireSize         1880465
LandOwnerType    1880465
FireState        1880465
dtype: int64

In [42]:
df_fire["LandOwnerType"].value_counts()
# Might have been an interesting field, but Too many Missing to be useful.

MISSING/NOT SPECIFIED    1050835
PRIVATE                   314822
USFS                      188338
BIA                       106819
STATE OR PRIVATE           71881
BLM                        63278
STATE                      30790
NPS                        17524
FWS                        12191
TRIBAL                      8952
OTHER FEDERAL               6452
MUNICIPAL/LOCAL             4236
UNDEFINED FEDERAL           2206
COUNTY                      1841
BOR                          285
FOREIGN                       15
Name: LandOwnerType, dtype: int64

In [41]:
df_fire["FireCause"].value_counts()
# Could use a bar chart for this see distribution

Debris Burning       429028
Miscellaneous        323805
Arson                281455
Lightning            278468
Missing/Undefined    166723
Equipment Use        147612
Campfire              76139
Children              61167
Smoking               52869
Railroad              33455
Powerline             14448
Fireworks             11500
Structure              3796
Name: FireCause, dtype: int64

**EDITING VALUES VIDEO - COREY SCHAFER - https://www.youtube.com/watch?v=DCDe29sIKcE**

In [37]:
df_fire["FireState"].value_counts()

CA    189550
GA    168867
TX    142021
NC    111277
FL     90261
SC     81315
NY     80870
MS     79230
AZ     71586
AL     66570
OR     61088
MN     44769
OK     43239
MT     40767
NM     37478
ID     36698
CO     34157
WA     33513
WI     31861
AR     31663
TN     31154
SD     30963
UT     30725
LA     30013
KY     27089
NJ     25949
PR     22081
WV     21967
VA     21833
MO     17953
NV     16956
ND     15201
WY     14166
ME     13150
AK     12843
MI     10502
HI      9895
PA      8712
NE      7973
KS      7673
CT      4976
IA      4134
MD      3622
OH      3479
MA      2626
NH      2452
IL      2327
IN      2098
RI       480
VT       456
DE       171
DC        66
Name: FireState, dtype: int64

In [10]:
# Create a new DataFrame with the unique values of the FireCause field.
df_cause_names = pd.DataFrame(df_fire["FireCause"].unique())

In [11]:
df_cause_names

,0
0,Miscellaneous
1,Lightning
2,Debris Burning
3,Campfire
4,Equipment Use
5,Arson
6,Children
7,Railroad
8,Smoking
9,Powerline


In [12]:
# Create a list of "Cause Catagories" in the order of Cause Names
cause_catagories=['Unknown', 'Natural', 'Industrial', 'Human Caused', 'Industrial', 'Human Caused', 'Human Caused', 'Industrial', 'Human Caused', 'Industrial', 'Industrial', 'Human Caused', 'Unknown']

In [13]:
cause_catagories

['Unknown',
 'Natural',
 'Industrial',
 'Human Caused',
 'Industrial',
 'Human Caused',
 'Human Caused',
 'Industrial',
 'Human Caused',
 'Industrial',
 'Industrial',
 'Human Caused',
 'Unknown']

In [14]:
# Add a new field to the Cause Names DataFrame, and load the Catagories to match the Names
df_cause_names['CauseCatagory'] = cause_catagories

In [15]:
df_cause_names

,0,CauseCatagory
0,Miscellaneous,Unknown
1,Lightning,Natural
2,Debris Burning,Industrial
3,Campfire,Human Caused
4,Equipment Use,Industrial
5,Arson,Human Caused
6,Children,Human Caused
7,Railroad,Industrial
8,Smoking,Human Caused
9,Powerline,Industrial


In [16]:
# Update the column Names of the Fire Causes DF: 
df_cause_names.columns = 'FireCause', 'CauseCatagory'

In [50]:
df_cause_names

,FireCause,CauseCatagory
0,Miscellaneous,Unknown
1,Lightning,Natural
2,Debris Burning,Industrial
3,Campfire,Human Caused
4,Equipment Use,Industrial
5,Arson,Human Caused
6,Children,Human Caused
7,Railroad,Industrial
8,Smoking,Human Caused
9,Powerline,Industrial


In [18]:
# Merge New Data Field into Main DataFrame
df_full_fire = pd.merge(df_fire, df_cause_names, how="inner", on=["FireCause", "FireCause"])

In [19]:
df_full_fire

,FireYear,DiscoveryDOY,FireCause,FireSize,LandOwnerType,FireState,CauseCatagory
0,2005,33,Miscellaneous,0.10,USFS,CA,Unknown
1,2004,247,Miscellaneous,0.10,USFS,CA,Unknown
2,2004,78,Miscellaneous,8.20,MISSING/NOT SPECIFIED,NM,Unknown
3,2005,70,Miscellaneous,10.00,MISSING/NOT SPECIFIED,NC,Unknown
4,2005,70,Miscellaneous,5.00,USFS,OR,Unknown
...,...,...,...,...,...,...,...
1880460,2015,255,Missing/Undefined,2.00,STATE OR PRIVATE,CA,Unknown
1880461,2015,165,Missing/Undefined,2.22,STATE OR PRIVATE,CA,Unknown
1880462,2015,269,Missing/Undefined,0.01,STATE OR PRIVATE,CA,Unknown
1880463,2015,122,Missing/Undefined,0.10,MUNICIPAL/LOCAL,CA,Unknown


In [60]:
fire_4charts = df_full_fire[['CauseCatagory','FireCause']]

In [61]:
fire_4charts

,CauseCatagory,FireCause
0,Unknown,Miscellaneous
1,Unknown,Miscellaneous
2,Unknown,Miscellaneous
3,Unknown,Miscellaneous
4,Unknown,Miscellaneous
...,...,...
1880460,Unknown,Missing/Undefined
1880461,Unknown,Missing/Undefined
1880462,Unknown,Missing/Undefined
1880463,Unknown,Missing/Undefined


In [62]:
# Add Field for Charts to Total
FireCount = 1

In [65]:
# Add Count Field to be used in Charts
fire_4charts['CountFires'] = FireCount

In [64]:
fire_4charts

,CauseCatagory,FireCause,CountFires
0,Unknown,Miscellaneous,1
1,Unknown,Miscellaneous,1
2,Unknown,Miscellaneous,1
3,Unknown,Miscellaneous,1
4,Unknown,Miscellaneous,1
...,...,...,...
1880460,Unknown,Missing/Undefined,1
1880461,Unknown,Missing/Undefined,1
1880462,Unknown,Missing/Undefined,1
1880463,Unknown,Missing/Undefined,1


In [69]:
fire_4charts.dtypes

CauseCatagory    object
FireCause        object
CountFires        int64
dtype: object

In [ ]:
fire_4charts = df.grade.astype(int)

## Add Tree Map Chart to Visualize Fire Causes and Cause Catagories

In [66]:
import pandas as pd
import plotly
import plotly.express as px

In [72]:
figure2=px.treemap(fire_4charts,
            path=['CauseCatagory', 'FireCause'],
            values='CountFires')

In [73]:
figure2

In [83]:
figure3 = px.treemap(
        data_frame = fire_4charts,
        path = ['CauseCatagory', 'FireCause'],
        values = 'CountFires',
        title = "    WILDFIRE DATABASE - CATEGORIES AND CAUSES OF 1.8M U.S. FIRES")

In [85]:
figure3.update_layout(
        title_font_size=20,
        title_font_family='Arial')

## Prepare Data for Prediction Models

In [54]:
df_new_fire = df_full_fire.drop(['LandOwnerType','FireCause'],axis='columns')

In [57]:
df_new_fire['CauseCatagory'].value_counts()

Industrial      628339
Unknown         490528
Human Caused    483130
Natural         278468
Name: CauseCatagory, dtype: int64

In [58]:
df_new_fire

,FireYear,DiscoveryDOY,FireSize,FireState,CauseCatagory
0,2005,33,0.10,CA,Unknown
1,2004,247,0.10,CA,Unknown
2,2004,78,8.20,NM,Unknown
3,2005,70,10.00,NC,Unknown
4,2005,70,5.00,OR,Unknown
...,...,...,...,...,...
1880460,2015,255,2.00,CA,Unknown
1880461,2015,165,2.22,CA,Unknown
1880462,2015,269,0.01,CA,Unknown
1880463,2015,122,0.10,CA,Unknown


In [59]:
df_dum = pd.get_dummies(df_new_fire.FireState)

In [60]:
df_dum

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1880461,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1880462,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1880463,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df_merged = pd.concat([df_new_fire,df_dum],axis='columns')

In [63]:
df_merged

,FireYear,DiscoveryDOY,FireSize,FireState,CauseCatagory,AK,AL,AR,AZ,CA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,2005,33,0.10,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2004,247,0.10,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2004,78,8.20,NM,Unknown,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2005,70,10.00,NC,Unknown,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2005,70,5.00,OR,Unknown,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,255,2.00,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1880461,2015,165,2.22,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1880462,2015,269,0.01,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1880463,2015,122,0.10,CA,Unknown,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [67]:
data_df = df_merged.drop(['CauseCatagory','FireState'],axis='columns')


In [68]:
data_df

,FireYear,DiscoveryDOY,FireSize,AK,AL,AR,AZ,CA,CO,CT,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,2005,33,0.10,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2004,247,0.10,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2004,78,8.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2005,70,10.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2005,70,5.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,2015,255,2.00,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1880461,2015,165,2.22,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1880462,2015,269,0.01,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1880463,2015,122,0.10,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
cause_df = df_merged['CauseCatagory']

In [70]:
cause_df

0          Unknown
1          Unknown
2          Unknown
3          Unknown
4          Unknown
            ...   
1880460    Unknown
1880461    Unknown
1880462    Unknown
1880463    Unknown
1880464    Unknown
Name: CauseCatagory, Length: 1880465, dtype: object

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_df,cause_df,test_size=0.2)

# Run Machine Learning Models

In [73]:
# Create and Fit LR Model, check Score: 
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.4780174052694413

In [ ]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [1]:
cur.close()
conn.close()